# Тема задачи: Аналитика EdTech‑продукта — воронка и A/B‑эксперимент

**Цель:** Проанализировать путь пользователя от регистрации до первой оплаты, оценить узкие места воронки и сравнить конверсию в оплату между группами A и B.

**Ключевые вопросы:**
1. Какова конверсия по этапам: `registration → start_lesson → payment`?
2. Где основные потери и почему?
3. Отличается ли доля оплат между группами A и B? Есть ли статистически значимая разница?
4. Какие гипотезы и продуктовые изменения стоит проверить в следующем цикле?


# Данные и допущения

**Наборы данных:**  
- `users.csv` — пользователи (user_id, registration_date, traffic_source, device).  
- `events.csv` — события продукта: *registration*, *login*, *start_lesson*, *payment* (user_id, event_time).  
- `lessons.csv` — прохождения уроков (user_id, lesson_id, lesson_time, ...).  
- `payments.csv` — оплаты (user_id, amount, payment_time, ...).  
- `ab_groups.csv` — разметка A/B (user_id, group).  

**Определения и метрики:**  
- **Регистрация** — событие `registration` в `events` (дополнительно валидируем по `users`).  
- **Старт урока** — событие `start_lesson` в `events`.  
- **Оплата** — наличие строки в `payments` (уникальные пользователи считаются плательщиками).  
- **Pay rate** — доля пользователей с оплатой среди зарегистрировавшихся.  
- **A/B** — сравнение метрик между группами `A` и `B` из `ab_groups` с предварительной проверкой **SRM** (chi-square).
- Для сравнения долей (например, pay rate) используем тест пропорций или chi-square (для простоты в демонстрации).

**Период данных:** будет определён автоматически в секции загрузки/подготовки данных (по мин/макс датам в CSV).

**Окружение:** `pandas`, `numpy`, `scipy`, при необходимости `matplotlib/plotly` для графиков.


# Тема задачи

**Аналитика EdTech‑продукта:**  
Необходимо проанализировать путь пользователя от регистрации до оплаты,  
построить воронку конверсии и проверить результаты A/B‑теста на основе предоставленных данных.

**Цель:**  
Выявить слабые места воронки (где больше всего теряется пользователей) и оценить,  
есть ли статистически значимые отличия в оплатах между группами A и B.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
users = pd.read_csv('users.csv')
events = pd.read_csv('events.csv')

In [ ]:
events.head()

In [ ]:
users.head()

In [ ]:
'''
Задание:
Построй таблицу, где для каждой когорты (дата регистрации) указано, сколько пользователей вернулись через 0, 1, 2, ..., 30 дней после регистрации.

SELECT
  u.registration_date AS cohort,
  e.event_date,
  (e.event_date - u.registration_date) AS day_diff,
  COUNT(DISTINCT u.user_id) AS retained_users
FROM users u
JOIN events e ON u.user_id = e.user_id
WHERE e.event_type = 'login'
  AND e.event_date >= u.registration_date
  AND e.event_date <= u.registration_date + INTERVAL '30 day'
GROUP BY cohort, day_diff
ORDER BY cohort, day_diff;
'''

In [ ]:
# Retention
df = events.merge(users, on='user_id')

# Приводим даты к datetime
df['event_date'] = pd.to_datetime(df['event_date'])
df['registration_date'] = pd.to_datetime(df['registration_date'])

df1 = df.copy()
# Считаем разницу
df1['day_diff'] = (df1['event_date'] - df1['registration_date']).dt.days

# Фильтруем Retention 0–30 дней по логинам
df1 = df1[(df1['event_type'] == 'login') & (df1['day_diff'] >= 0) & (df1['day_diff'] <= 30)]

# Строим pivot
df_pivot = df1.pivot_table(
    values='user_id',
    index='registration_date',
    columns='day_diff',
    aggfunc='nunique',
    fill_value=0
)

df_pivot

In [ ]:
# Воронка
# 1. Получим только тех, кто зарегистрировался
registered_users = df[df['event_type'] == 'registration']['user_id'].unique()

# 2. Оставим только события этих пользователей
df_filtered = df[df['user_id'].isin(registered_users)]

steps = ['registration', 'login', 'start_lesson', 'payment']

funnel = (
    df_filtered[df_filtered['event_type'].isin(steps)]
    .groupby(['event_type'])['user_id']
    .nunique()
    .reindex(steps)
    .reset_index()
    .rename(columns={'user_id': 'users'})
)
# 4. Добавим конверсию
funnel['conversion_%'] = round(funnel['users'] / funnel['users'].iloc[0] * 100, 1)
funnel

In [ ]:
# a/b test

ab_group = pd.read_csv('ab_groups.csv')
df_ab = df.merge(ab_group,on='user_id',how='inner')
df_ab['is_payer'] = (df_ab['event_type'] == 'payment').astype(int)
df_user_level = df_ab.groupby(['user_id','group'])['is_payer'].max().reset_index()
df_cr = df_user_level.groupby('group')['is_payer'].mean().round(3).reset_index()
group_a = df_user_level[df_user_level['group'] == 'A']['is_payer']
group_b = df_user_level[df_user_level['group'] == 'B']['is_payer']
t_stat,p_value = stats.ttest_ind(group_a,group_b,equal_var=False)
print('Conversion rate: \n', df_cr)
print()
print('t-статистика:', round(t_stat, 3))
print('p-значение:', round(p_value, 4))

In [ ]:
from statsmodels.stats.power import NormalIndPower
import numpy as np

baseline = 0.273
n_per_group = 2500
alpha = 0.05
power = 0.8

power_analysis = NormalIndPower()
effect_size = power_analysis.solve_power(
    effect_size=None, 
    nobs1=n_per_group,
    alpha=alpha,
    power=power,
    ratio=1.0,
    alternative='two-sided'
)

def cohen_h_inverse(h, p1):
    return (np.sin(np.arcsin(np.sqrt(p1)) + h / 2))**2

p2 = cohen_h_inverse(effect_size, baseline)
mde = round((p2 - baseline) * 100, 2)

print(f" Минимально значимая разница (MDE): {mde} п.п.")
print(f" Целевая конверсия: {round(p2 * 100, 2)}%")

# Анализ воронки регистрации и причины низкой оплаты

## Проблема
На этапе оплаты остаётся лишь каждый 4‑й пользователь, начавший урок.

## Возможные причины
Низкая воспринимаемая ценность:

Пользователь не увидел, зачем платить после первого урока.

Возможно, уроки дают мало результата или неполно показывают продукт.

Сложность в оплате:

Долгий путь до кнопки “Оплатить”

Нет популярных способов оплаты (например, Kaspi, рассрочка, PayPal и др.)

Нет системы возвращения пользователя:

После начала урока пользователь отвлёкся и не вернулся

Не настроены напоминания, нет прогрева через рассылки

## Гипотезы для проверки
Гипотеза
1. Добавим CTA и преимущества после урока
2. Упростим путь до оплаты
3. Включим пуши/email тем, кто начал, но не оплатил

Как проверим
1. A/B‑тест: экран с аргументами vs без
2. A/B‑тест с быстрым переходом на оплату
3. Сравнение CR между пользователями с и без пушей

## Цель эксперимента:
Проверить гипотезу о росте конверсии оплаты с 27.3% до минимум 30.9% (+3.6 п.п.), 
что соответствует MDE при текущем трафике (5000 пользователей).
Желаемый ориентир на рынке — 35%, к которому мы стремимся в перспективе.

In [ ]:
plt.figure(figsize=(10,6))
bars = plt.bar(funnel['event_type'], funnel['users'], color='skyblue')

for bar, conv in zip(bars, funnel['conversion_%']):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 20, f'{int(yval)}\n({conv}%)', 
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.title('Воронка пользовательского пути', fontsize=14)
plt.xlabel('Этапы', fontsize=12)
plt.ylabel('Количество пользователей', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(0, funnel['users'].max()*1.2)
plt.tight_layout()
plt.show()